# 🤖 Comparación ToT vs RAG con JuezIA
Este notebook permite comparar los métodos de generación **Tree-of-Thoughts (ToT)** y **RAG (Retrieval-Augmented Generation)** usando `gemma3:1b` vía Ollama, con evaluación automática por un modelo juez (`JuezIA`).

In [ ]:
# Configuración de entorno para imports relativos
import sys
from pathlib import Path
BASE_DIR = Path("..").resolve()
SRC_DIR = BASE_DIR / "src"
sys.path.append(str(BASE_DIR))
sys.path.append(str(SRC_DIR))

from chatbot import Chatbot
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from datetime import datetime
import json

In [ ]:
# Inicializar chatbot y juez (gemma3 como tutor y juez)
tutor = Chatbot("Eres un tutor inteligente que razona paso a paso en español.")
juez = Chatbot("Eres un crítico educativo. Evalúa si la respuesta del tutor es clara, precisa y útil.")

In [ ]:
# Tema a comparar
tema = "¿Por qué es importante el análisis matemático en ciencia de datos?"

In [ ]:
# Simulación de ToT clásico (simplificado para demostración)
pensamientos = [
    tutor(f"Propón una idea sobre: {tema}"),
    tutor(f"Otra idea sobre: {tema}"),
    tutor(f"Una última idea sobre: {tema}")
]
print("Pensamientos generados (ToT):\n")
for i, p in enumerate(pensamientos):
    print(f"{i}. {p}\n")

seleccion = int(input("Selecciona el mejor pensamiento (índice): "))
mejor_pensamiento = pensamientos[seleccion]
respuesta_tot = tutor(f"Redacta una respuesta basada en esta idea: {mejor_pensamiento}")
print("\n📘 Respuesta ToT:\n", respuesta_tot)

In [ ]:
# Simulación de RAG con base embebida
raw_docs = [
    "El análisis matemático proporciona herramientas fundamentales para el modelado y la optimización de algoritmos en ciencia de datos.",
    "Sin cálculo diferencial e integral, sería imposible entender cómo cambian los valores en una red neuronal.",
    "Los fundamentos de álgebra lineal son esenciales para el manejo de vectores, matrices y transformaciones en espacios de alta dimensión.",
    "Estadística y probabilidad también se consideran parte del análisis matemático aplicado, especialmente en modelos predictivos."
]

embedding = OllamaEmbeddings(model="gemma:2b")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
docs = [Document(page_content=chunk) for doc in raw_docs for chunk in text_splitter.split_text(doc)]
vectorstore = Chroma.from_documents(documents=docs, embedding=embedding, persist_directory=".rag_comparacion_index")

docs_recuperados = vectorstore.similarity_search(tema, k=3)
contexto_rag = "\n\n".join([doc.page_content for doc in docs_recuperados])
prompt_rag = f"Contexto:\n{contexto_rag}\n\nPregunta:\n{tema}"
respuesta_rag = tutor(prompt_rag)
print("\n📘 Respuesta RAG:\n", respuesta_rag)

In [ ]:
# Evaluación automática con JuezIA
evaluacion_tot = juez(f"Pregunta: {tema}\nRespuesta del tutor: {respuesta_tot}\n¿La explicación es clara y útil? Justifica.")
evaluacion_rag = juez(f"Pregunta: {tema}\nRespuesta del tutor: {respuesta_rag}\n¿La explicación es clara y útil? Justifica.")

print("\n⚖️ Evaluación ToT:\n", evaluacion_tot)
print("\n⚖️ Evaluación RAG:\n", evaluacion_rag)

In [ ]:
# Guardar comparación
resultado = {
    "tema": tema,
    "tot": {
        "respuesta": respuesta_tot,
        "evaluacion": evaluacion_tot
    },
    "rag": {
        "respuesta": respuesta_rag,
        "evaluacion": evaluacion_rag
    },
    "timestamp": datetime.now().isoformat()
}

filename = f"comparacion_tot_vs_rag_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(filename, "w", encoding="utf-8") as f:
    json.dump(resultado, f, indent=2, ensure_ascii=False)

print(f"\n✅ Resultado guardado en {filename}")